In [ ]:
!pip install accelerate -U
!pip install transformers -U
!pip install bitsandbytes deepspeed wandb peft
!pip install mpi4py
!pip install flash-attn --no-build-isolation

In [14]:
!git clone https://github.com/rsk2327/LLaVA/

!mkdir -p /content/checkpoints/llava-v1.5-13b-pretrain
!mkdir -p /content/checkpoints/llava-v1.5-13b-pretrain/logs

!mkdir -p /content/checkpoints/llava-v1.5-7b-pretrain
!mkdir -p /content/checkpoints/llava-v1.5-7b-pretrain/logs

%load_ext autoreload
%autoreload 2


Cloning into 'LLaVA'...
remote: Enumerating objects: 2314, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 2314 (delta 24), reused 41 (delta 13), pack-reused 2255
Receiving objects: 100% (2314/2314), 13.75 MiB | 7.96 MiB/s, done.
Resolving deltas: 100% (1400/1400), done.


### **DATA SETUP**

In [ ]:
##### Download projector weights
!wget https://huggingface.co/liuhaotian/llava-v1.5-mlp2x-336px-pretrain-vicuna-13b-v1.5/resolve/main/mm_projector.bin

!mv mm_projector.bin /content/checkpoints/llava-v1.5-13b-pretrain/

!wget https://huggingface.co/liuhaotian/llava-v1.5-mlp2x-336px-pretrain-vicuna-7b-v1.5/resolve/main/mm_projector.bin

!mv mm_projector.bin /content/checkpoints/llava-v1.5-7b-pretrain/

#### Downloading Test PALO JSON file
!wget https://huggingface.co/datasets/roshansk23/palo_coco_subset/resolve/main/palo_multilingual_dataset_1k.json

!mv palo_multilingual_dataset_1k.json /content/LLaVA/playground/data/


#### Downloading PALO Validation subset
!wget http://images.cocodataset.org/zips/val2017.zip

!mkdir -p /content/LLaVA/playground/data/PALO

!unzip /content/val2017.zip

!mv /content/val2017/ /content/LLaVA/playground/data/PALO/


In [8]:
#### Modifying the PALO label file
# For testing, we modify the palo label data to match images from the COCO validation data
imagefilelist = os.listdir('/content/LLaVA/playground/data/PALO/val2017')

data = json.load(open('/content/LLaVA/playground/data/palo_multilingual_dataset_1k.json'))

for i in tqdm(range(len(data))):

    id = imagefilelist[i].split(".")[0]
    image = 'PALO/val2017/' + imagefilelist[i]

    data[i]['id'] = id
    data[i]['image'] = image


with open('palo_multilingual_dataset_1k_val2017.json', 'w') as f:
    json.dump(data, f)

!mv palo_multilingual_dataset_1k_val2017.json /content/LLaVA/playground/data/

100%|██████████| 1000/1000 [00:00<00:00, 822090.16it/s]


In [17]:
import transformers
import os
import torch
from tqdm import tqdm
import json
import pathlib

import sys
sys.path.insert(0,'/content/LLaVA/')

os.chdir('/content/LLaVA/playground/finetuning')

from finetune_args import *
from llava.model import *
from llava import conversation as conversation_lib
from dataset_utils import *

from llava.train.llava_trainer import LLaVATrainer
from llava.mm_utils import tokenizer_image_token
from llava.constants import IGNORE_INDEX, IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN

In [4]:
#### Training Arguments


# model_args, data_args, training_args = get_finetune_args(
#                                                         # model_name = 'lmsys/vicuna-13b-v1.5',
#                                                         # model_name = 'liuhaotian/llava-v1.5-7b',
#                                                         model_name = 'lmsys/vicuna-7b-v1.5',
#                                                         pretrain_mm_mlp_adapter = '/content/checkpoints/llava-v1.5-7b-pretrain/mm_projector.bin',
#                                                         data_path = '/content/LLaVA/playground/data/palo_multilingual_dataset_1k_val2017.json',
#                                                         image_folder = '/content/LLaVA/playground/data/',
#                                                         output_dir = '/content/checkpoints/llava-v1.5-7b/',
#                                                         logging_dir = '/content/checkpoints/llava-v1.5-7b/logs/'
#                                                         )


## For LORA version
model_args, data_args, training_args = get_finetune_lora_args(
                                                        # model_name = 'lmsys/vicuna-13b-v1.5',
                                                        # model_name = 'liuhaotian/llava-v1.5-7b',
                                                        model_name = 'lmsys/vicuna-7b-v1.5',
                                                        pretrain_mm_mlp_adapter = '/content/checkpoints/llava-v1.5-7b-pretrain/mm_projector.bin',
                                                        data_path = '/content/LLaVA/playground/data/palo_multilingual_dataset_1k_val2017.json',
                                                        image_folder = '/content/LLaVA/playground/data/',
                                                        output_dir = '/content/checkpoints/llava-v1.5-7b/',
                                                        logging_dir = '/content/checkpoints/llava-v1.5-7b/logs/'
                                                        )

attn_implementation="flash_attention_2"
bnb_model_from_pretrained_args = {}

training_args.per_device_train_batch_size = 4
training_args.gradient_accumulation_steps = 4

local_rank = None

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## **MODEL SETUP**

In [5]:
model = LlavaLlamaForCausalLM.from_pretrained(
                model_args.model_name_or_path,
                cache_dir=training_args.cache_dir,
                attn_implementation=attn_implementation,
                torch_dtype=(torch.bfloat16 if training_args.bf16 else None),
                **bnb_model_from_pretrained_args
            )

You are using a model of type llama to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [18]:
model.config.use_cache = False

if model_args.freeze_backbone:
    model.model.requires_grad_(False)

if training_args.gradient_checkpointing:
    if hasattr(model, "enable_input_require_grads"):
        model.enable_input_require_grads()
    else:
        def make_inputs_require_grad(module, input, output):
            output.requires_grad_(True)
        model.get_input_embeddings().register_forward_hook(make_inputs_require_grad)

if training_args.lora_enable:
    from peft import LoraConfig, get_peft_model
    lora_config = LoraConfig(
        r=training_args.lora_r,
        lora_alpha=training_args.lora_alpha,
        target_modules=find_all_linear_names(model),
        lora_dropout=training_args.lora_dropout,
        bias=training_args.lora_bias,
        task_type="CAUSAL_LM",
    )
    if training_args.bits == 16:
        if training_args.bf16:
            model.to(torch.bfloat16)
        if training_args.fp16:
            model.to(torch.float16)
    model = get_peft_model(model, lora_config)

tokenizer = transformers.AutoTokenizer.from_pretrained(
            model_args.model_name_or_path,
            cache_dir=training_args.cache_dir,
            model_max_length=training_args.model_max_length,
            padding_side="right",
            use_fast=False,
        )

tokenizer.pad_token = tokenizer.unk_token
if model_args.version in conversation_lib.conv_templates:
    conversation_lib.default_conversation = conversation_lib.conv_templates[model_args.version]
else:
    conversation_lib.default_conversation = conversation_lib.conv_templates["vicuna_v1"]

In [19]:
model.get_model()

LlavaLlamaModel(
  (embed_tokens): Embedding(32000, 4096, padding_idx=0)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaFlashAttention2(
        (q_proj): lora.Linear(
          (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.05, inplace=False)
          )
          (lora_A): ModuleDict(
            (default): Linear(in_features=4096, out_features=128, bias=False)
          )
          (lora_B): ModuleDict(
            (default): Linear(in_features=128, out_features=4096, bias=False)
          )
          (lora_embedding_A): ParameterDict()
          (lora_embedding_B): ParameterDict()
        )
        (k_proj): lora.Linear(
          (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
          (lora_dropout): ModuleDict(
            (default): Dropout(p=0.05, inplace=False)
          )
          (lora_A): ModuleDict(
            (defau

In [21]:
training_args.freeze_mm_mlp_adapter

False

In [22]:
model.get_model().initialize_vision_modules(
            model_args=model_args,
            fsdp=training_args.fsdp
        )

vision_tower = model.get_vision_tower()
vision_tower.to(dtype=torch.bfloat16 if training_args.bf16 else torch.float16, device=training_args.device)

data_args.image_processor = vision_tower.image_processor
data_args.is_multimodal = True

model.config.image_aspect_ratio = data_args.image_aspect_ratio
model.config.tokenizer_padding_side = tokenizer.padding_side
model.config.tokenizer_model_max_length = tokenizer.model_max_length

model.config.tune_mm_mlp_adapter = training_args.tune_mm_mlp_adapter = model_args.tune_mm_mlp_adapter

# if model_args.tune_mm_mlp_adapter:
#     model.requires_grad_(False)
#     for p in model.get_model().mm_projector.parameters():
#         p.requires_grad = True

model.config.freeze_mm_mlp_adapter = training_args.freeze_mm_mlp_adapter
# if training_args.freeze_mm_mlp_adapter:
#     for p in model.get_model().mm_projector.parameters():
#         p.requires_grad = False

# if training_args.bits in [4, 8]:
#     model.get_model().mm_projector.to(dtype=compute_dtype, device=training_args.device)

model.config.mm_use_im_start_end = data_args.mm_use_im_start_end = model_args.mm_use_im_start_end
model.config.mm_projector_lr = training_args.mm_projector_lr
training_args.use_im_start_end = model_args.mm_use_im_start_end
model.config.mm_use_im_patch_token = model_args.mm_use_im_patch_token
model.initialize_vision_tokenizer(model_args, tokenizer=tokenizer)

# if training_args.bits in [4, 8]:
#     from peft.tuners.lora import LoraLayer
#     for name, module in model.named_modules():
#         if isinstance(module, LoraLayer):
#             if training_args.bf16:
#                 module = module.to(torch.bfloat16)
#         if 'norm' in name:
#             module = module.to(torch.float32)
#         if 'lm_head' in name or 'embed_tokens' in name:
#             if hasattr(module, 'weight'):
#                 if training_args.bf16 and module.weight.dtype == torch.float32:
#                     module = module.to(torch.bfloat16)

data_module = make_supervised_data_module(tokenizer=tokenizer,
                                              data_args=data_args)

trainer = LLaVATrainer(model=model,
                    tokenizer=tokenizer,
                    args=training_args,
                    **data_module)

In [23]:
if list(pathlib.Path(training_args.output_dir).glob("checkpoint-*")):
    trainer.train(resume_from_checkpoint=True)
else:
    trainer.train()

[2024-07-18 13:01:31,219] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: roshan-santhosh (roshan-santhosh-meta). Use `wandb login --relogin` to force relogin


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2452 > 2048). Running this sequence through

Step,Training Loss
1,1.191000
2,1.336300
3,1.279300
4,1.115900
5,1.158900
6,1.136200
7,1.440000
8,1.076700
9,1.079900
10,1.100400


Token indices sequence length is longer than the specified maximum sequence length for this model (2087 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2074 > 2048). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2385 > 2048). Running this sequence through the model will result in indexing errors


Step,Training Loss
1,1.191000
2,1.336300
3,1.279300
4,1.115900
5,1.158900
6,1.136200
7,1.440000
8,1.076700
9,1.079900
10,1.100400


AttributeError: 'LlavaLlamaForCausalLM' object has no attribute 'save_checkpoint'

1

In [ ]:
# projector = torch.load('/content/checkpoints/llava-v1.5-13b-pretrain/mm_projector.bin', map_location='cpu')

# projector['model.mm_projector.0.weight'].shape
